In [1]:
import os
%pwd

'e:\\Courses\\Projects\\E2E_DeepLearning_MLFlow_DVC_CICD\\research'

In [2]:
os.chdir('../')
%pwd

'e:\\Courses\\Projects\\E2E_DeepLearning_MLFlow_DVC_CICD'

In [3]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/tripathivishwadeepak/E2E_DeepLearning_MLFlow_DVC_CICD.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "tripathivishwadeepak"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "c5e1c9532cc301ea889416e189cae21fc1cceb7b"

In [ ]:
import dagshub
dagshub.init(repo_owner='tripathivishwadeepak', repo_name='E2E_DeepLearning_MLFlow_DVC_CICD', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

In [4]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/model_training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/model_training/model.h5",
            training_data = "artifacts/data_ingestion/CTScan",
            mlflow_uri = "https://dagshub.com/tripathivishwadeepak/E2E_DeepLearning_MLFlow_DVC_CICD.mlflow",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config


In [12]:
from pathlib import Path
import tensorflow as tf 
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self,config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    #(use staticmethod is if you have something that could be written as a standalone function (not part of any class),
    # but you want to keep it within the class because it's somehow semantically related to the class.)
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy":self.score[1]}
        save_json(path = Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_score = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss":self.score[0],"accuracy":self.score[1]}
            )
            # Model registry doesnot work with file store
            if tracking_url_type_score != "file":
                mlflow.keras.log_model(self.model,"model",registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model,"model")

In [24]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-03-26 17:44:05,774 INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-26 17:44:05,833 INFO: common: yaml file: params.yaml loaded successfully]
Found 2207 images belonging to 2 classes.
138/138 [==============================] - 1297s 9s/step - loss: 14.4403 - accuracy: 0.6901
[2025-03-26 18:06:52,814 WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)': /tripathivishwadeepak/E2E_DeepLearning_MLFlow_DVC_CICD.mlflow/api/2.0/mlflow/runs/create]


2025/03/26 18:06:57 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-03-26 18:07:29,564 WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\VISHWA~1\AppData\Local\Temp\tmpwx2vymuq\model\data\model\assets
[2025-03-26 18:07:35,487 INFO: builder_impl: Assets written to: C:\Users\VISHWA~1\AppData\Local\Temp\tmpwx2vymuq\model\data\model\assets]


2025/03/26 18:10:56 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\VISHWA~1\AppData\Local\Temp\tmpwx2vymuq\model, flavor: tensorflow), fall back to return ['tensorflow==2.12.0']. Set logging level to DEBUG to see the full traceback.
e:\Courses\Anaconda\envs\cnnClassifier\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


[2025-03-26 18:13:27,630 WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)': /tripathivishwadeepak/E2E_DeepLearning_MLFlow_DVC_CICD.mlflow/api/2.0/mlflow-artifacts/artifacts/9b0ae64153274840bb7b576316876846/b679eea75bec477b874bad84ba9eee66/artifacts/model/data/model/variables/variables.data-00000-of-00001]


: 